# SMA Crossover Strategy
Author: TraderPy

Youtube: https://www.youtube.com/channel/UC9xYCyyR_G3LIuJ_LlTiEVQ/featured

Website: https://traderpy.com/

## Disclaimer
Trading the financial markets imposes a risk of financial loss. TraderPy is not responsible for any financial losses that viewers suffer. Content is educational only and does not serve as financial advice. Information or material is provided ‘as is’ without any warranty. 

Past trading results do not indicate future performance. Strategies that worked in the past may not reflect the same results in the future.

In [416]:
import MetaTrader5 as mt5
import pandas as pd
import plotly.express as px
import numpy as np

In [417]:
# connect to MetaTrader5 as mt5
mt5.initialize()

True

In [418]:
# settings
symbol = 'EURUSD'
timeframe = mt5.TIMEFRAME_D1
start_pos = 0
num_bars = 1000

fast_sma_period = 10
slow_sma_period = 100

In [419]:
# Requesting historical data
bars = mt5.copy_rates_from_pos(symbol, timeframe, start_pos, num_bars)

df = pd.DataFrame(bars)[['time', 'open', 'high', 'low', 'close', 'spread']]
df['time'] = pd.to_datetime(df['time'], unit='s')

df['fast_sma'] = df['close'].rolling(fast_sma_period).mean()
df['slow_sma'] = df['close'].rolling(slow_sma_period).mean()

# finding crossovers
df['prev_fast_sma'] = df['fast_sma'].shift(1)

df.dropna(inplace=True)
df

,time,open,high,low,close,spread,fast_sma,slow_sma,prev_fast_sma
99,2020-07-24,1.15959,1.16568,1.15805,1.16549,2,1.147574,1.108153,1.143990
100,2020-07-27,1.16421,1.17803,1.16408,1.17510,2,1.151649,1.108458,1.147574
101,2020-07-28,1.17512,1.17727,1.16978,1.17150,2,1.154806,1.108882,1.151649
102,2020-07-29,1.17148,1.18051,1.17118,1.17906,3,1.158598,1.109412,1.154806
103,2020-07-30,1.17910,1.18480,1.17299,1.18467,2,1.163231,1.110092,1.158598
...,...,...,...,...,...,...,...,...,...
995,2024-01-05,1.09464,1.09974,1.08757,1.09401,15,1.100139,1.075829,1.100150
996,2024-01-08,1.09448,1.09777,1.09217,1.09499,15,1.099529,1.075908,1.100139
997,2024-01-09,1.09515,1.09653,1.09096,1.09298,15,1.098712,1.075967,1.099529
998,2024-01-10,1.09259,1.09719,1.09219,1.09710,15,1.098015,1.076044,1.098712


In [420]:
# Visualize Close Price
px.line(df, x='time', y='close')

In [421]:
def find_crossover(fast_sma, prev_fast_sma, slow_sma):
    
    if fast_sma > slow_sma and prev_fast_sma < slow_sma:
        return 'bullish crossover'
    elif fast_sma < slow_sma and prev_fast_sma > slow_sma:
        return 'bearish crossover'
    
    return None


df['crossover'] = np.vectorize(find_crossover)(df['fast_sma'], df['prev_fast_sma'], df['slow_sma'])

# df.dropna(inplace=True)
# signal = df[df['crossover'] == 'bullish crossover'].copy() # => Original
signal = df[(df['crossover'] == 'bullish crossover') | (df['crossover'] == 'bearish crossover')].copy()

signal

,time,open,high,low,close,spread,fast_sma,slow_sma,prev_fast_sma,crossover
259,2021-03-09,1.18447,1.19147,1.18346,1.18992,16,1.202320,1.203199,1.204812,bearish crossover
296,2021-04-29,1.21230,1.21490,1.21009,1.21172,20,1.206023,1.205323,1.204496,bullish crossover
334,2021-06-22,1.19159,1.19515,1.18800,1.19399,16,1.203132,1.203200,1.205451,bearish crossover
335,2021-06-23,1.19355,1.19693,1.19106,1.19247,16,1.200612,1.203090,1.203132,bearish crossover
698,2022-11-14,1.03334,1.03578,1.02704,1.03250,18,1.003800,1.002822,0.999376,bullish crossover
836,2023-05-25,1.07504,1.07557,1.07065,1.07242,10,1.080501,1.081419,1.082415,bearish crossover
853,2023-06-19,1.09359,1.09454,1.09063,1.09216,10,1.081020,1.080652,1.078920,bullish crossover
897,2023-08-18,1.08665,1.08929,1.08441,1.08712,10,1.092793,1.093049,1.094180,bearish crossover
964,2023-11-21,1.09384,1.09641,1.08990,1.09111,10,1.080956,1.079108,1.078828,bullish crossover


In [422]:
# visualize close price
fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

for i, row in signal.iterrows():
    fig.add_vline(x=row.time)
    
fig.show()

In [423]:
# creating backtest and position classes

class Position:
    def __init__(self, open_datetime, open_price, order_type, volume, lot_multiplier, sl, tp):
        self.open_datetime = open_datetime
        self.open_price = open_price
        self.order_type = order_type
        self.lot_multiplier = lot_multiplier
        self.volume = volume
        self.sl = sl
        self.tp = tp
        self.close_datetime = None
        self.close_price = None
        self.profit = None
        self.status = 'open'
        
    def close_position(self, close_datetime, close_price):
        self.close_datetime = close_datetime
        self.close_price = close_price
        self.profit = (self.close_price - self.open_price) * self.volume * self.lot_multiplier if self.order_type == 'buy' \
                                                                        else (self.open_price - self.close_price) * self.volume * self.lot_multiplier
        self.status = 'closed'
        
    def _asdict(self):
        return {
            'open_datetime': self.open_datetime,
            'open_price': self.open_price,
            'order_type': self.order_type,
            'volume': self.volume,
            'lot_multiplier': self.lot_multiplier,
            'sl': self.sl,
            'tp': self.tp,
            'close_datetime': self.close_datetime,
            'close_price': self.close_price,
            'profit': self.profit,
            'status': self.status,
        }
        
        
class Strategy:
    def __init__(self, df, starting_balance, volume, lot_multiplier):
        self.starting_balance = starting_balance
        self.positions = []
        self.lot_multiplier = lot_multiplier
        self.volume = volume
        self.data = df
        
    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df
        
    def add_position(self, position):
        self.positions.append(position)
        
        return True
        
# logic
    def run(self):
        for i, data in self.data.iterrows():
            
            if data.crossover == 'bearish crossover':
                for position in self.positions:
                    if position.status == 'open':
                        position.close_position(data.time, data.close)
                self.add_position(Position(data.time, data.close, 'sell', self.volume, 1, 0, 0))
            
            if data.crossover == 'bullish crossover':
                for position in self.positions:
                    if position.status == 'open':
                        position.close_position(data.time, data.close)
                self.add_position(Position(data.time, data.close, 'buy', self.volume, 1, 0, 0))
        
        return self.get_positions_df()

In [424]:
# Lot Size:
# Standard => Volume = 100,000 => Lot = 1.00 => pip Value = 10.00 USD
# Mini Lot => Volume = 10,000 => Lot = 0.10 => pip Value = 1.00 USD
# Micro Lot => Volume = 1,000 => Lot = 0.01 => pip Value = 0.10 USD
# Nano Lot => Volume = 100 => Lot = 0.0001 => pip Value = 10 USD

starting_balance = 1000 
standard_contract = 100000
min_lot = 0.1
lot_multiplier = 2 # 1 if not self.positions or self.positions[-1] > 0 else 2
volume = min_lot * standard_contract * lot_multiplier


In [425]:
sma_crossover_strategy = Strategy(df, starting_balance, volume, lot_multiplier)
deals = sma_crossover_strategy.run()
deals.dropna(inplace=True)
deals

,open_datetime,open_price,order_type,volume,lot_multiplier,sl,tp,close_datetime,close_price,profit,status,pnl
0,2021-03-09,1.18992,sell,20000.0,1,0,0,2021-04-29,1.21172,-436.0,closed,564.0
1,2021-04-29,1.21172,buy,20000.0,1,0,0,2021-06-22,1.19399,-354.6,closed,209.4
2,2021-06-22,1.19399,sell,20000.0,1,0,0,2021-06-23,1.19247,30.4,closed,239.8
3,2021-06-23,1.19247,sell,20000.0,1,0,0,2022-11-14,1.03250,3199.4,closed,3439.2
4,2022-11-14,1.03250,buy,20000.0,1,0,0,2023-05-25,1.07242,798.4,closed,4237.6
5,2023-05-25,1.07242,sell,20000.0,1,0,0,2023-06-19,1.09216,-394.8,closed,3842.8
6,2023-06-19,1.09216,buy,20000.0,1,0,0,2023-08-18,1.08712,-100.8,closed,3742.0
7,2023-08-18,1.08712,sell,20000.0,1,0,0,2023-11-21,1.09111,-79.8,closed,3662.2


In [426]:
#  Get Columns Titles to check Existing Columns
titles = deals.columns

for title in titles:
 print(title)


open_datetime
open_price
order_type
volume
lot_multiplier
sl
tp
close_datetime
close_price
profit
status
pnl


In [427]:
lot_multiplier = 2
deals['profit_1'] = deals['profit'].shift(1).fillna(0)
deals
# if deals['profit'].shift(-1) > 0 or deals['profit'].shift(-1) == None:
#     deals['lot'] = 0.1
# else:
#     deals['lot'] = 0.1 * lot_multiplier

,open_datetime,open_price,order_type,volume,lot_multiplier,sl,tp,close_datetime,close_price,profit,status,pnl,profit_1
0,2021-03-09,1.18992,sell,20000.0,1,0,0,2021-04-29,1.21172,-436.0,closed,564.0,0.0
1,2021-04-29,1.21172,buy,20000.0,1,0,0,2021-06-22,1.19399,-354.6,closed,209.4,-436.0
2,2021-06-22,1.19399,sell,20000.0,1,0,0,2021-06-23,1.19247,30.4,closed,239.8,-354.6
3,2021-06-23,1.19247,sell,20000.0,1,0,0,2022-11-14,1.03250,3199.4,closed,3439.2,30.4
4,2022-11-14,1.03250,buy,20000.0,1,0,0,2023-05-25,1.07242,798.4,closed,4237.6,3199.4
5,2023-05-25,1.07242,sell,20000.0,1,0,0,2023-06-19,1.09216,-394.8,closed,3842.8,798.4
6,2023-06-19,1.09216,buy,20000.0,1,0,0,2023-08-18,1.08712,-100.8,closed,3742.0,-394.8
7,2023-08-18,1.08712,sell,20000.0,1,0,0,2023-11-21,1.09111,-79.8,closed,3662.2,-100.8


In [428]:
#  Get total profit
total_profit = deals['profit'].sum()

total_profit

2662.200000000001

In [429]:
px.line(deals, x='close_datetime', y='pnl')

In [430]:
# visualize close price
fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

for i, row in signal.iterrows():
    fig.add_vline(x=row.time)
    
for i, row in deals[deals['status'] == 'closed'].iterrows():
    
    if row.profit > 0:
        fig.add_shape(type="line",
            x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
            line=dict(color="Green",width=3)
                     )
                      
    elif row.profit < 0:
        fig.add_shape(type="line",
            x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
            line=dict(color="Red",width=3)
                      )

    
fig.show()

In [431]:
def apply_profit_type(x):
# Finding profit type
  if x['profit'] > 0: 
      return "win"
  elif x['profit'] < 0:
      return "loss"
  else: return 'pass'

deals['profit_type'] = deals.apply(apply_profit_type, axis=1)

deals.dropna(inplace=True)

display(deals)

,open_datetime,open_price,order_type,volume,lot_multiplier,sl,tp,close_datetime,close_price,profit,status,pnl,profit_1,profit_type
0,2021-03-09,1.18992,sell,20000.0,1,0,0,2021-04-29,1.21172,-436.0,closed,564.0,0.0,loss
1,2021-04-29,1.21172,buy,20000.0,1,0,0,2021-06-22,1.19399,-354.6,closed,209.4,-436.0,loss
2,2021-06-22,1.19399,sell,20000.0,1,0,0,2021-06-23,1.19247,30.4,closed,239.8,-354.6,win
3,2021-06-23,1.19247,sell,20000.0,1,0,0,2022-11-14,1.03250,3199.4,closed,3439.2,30.4,win
4,2022-11-14,1.03250,buy,20000.0,1,0,0,2023-05-25,1.07242,798.4,closed,4237.6,3199.4,win
5,2023-05-25,1.07242,sell,20000.0,1,0,0,2023-06-19,1.09216,-394.8,closed,3842.8,798.4,loss
6,2023-06-19,1.09216,buy,20000.0,1,0,0,2023-08-18,1.08712,-100.8,closed,3742.0,-394.8,loss
7,2023-08-18,1.08712,sell,20000.0,1,0,0,2023-11-21,1.09111,-79.8,closed,3662.2,-100.8,loss


In [432]:
# Find Initial Deposit = Initial Balance 
deposit = starting_balance
print('deposit = ', deposit)

# find total net profit
total_net_profit = round(deals['profit'].sum())
print('total net profit = ', total_net_profit)

# find gross profit
gross_profit = round(sum(deals[deals['profit_type']=='win']['profit']))
print('gross profit = ', gross_profit)

# find gross loss
gross_loss = round(sum(deals[deals['profit_type']=='loss']['profit']))
print('gross loss = ', gross_loss)

#  find profit factor
profit_factor = gross_profit / gross_loss
print('profit factor = ', profit_factor)

# find largest profit trade
largest_profit_trade = round(deals['profit'].max())
print('largest profit trade = ', largest_profit_trade)

# find largest loss trade
largest_loss_trade = round(deals['profit'].min())
print('largest loss trade = ', largest_loss_trade)

deposit =  1000
total net profit =  2662
gross profit =  4028
gross loss =  -1366
profit factor =  -2.9487554904831623
largest profit trade =  3199
largest loss trade =  -436


In [433]:
out = (
    deals.groupby(deals["profit_type"].ne(deals["profit_type"].shift()).cumsum(), as_index=False)
    .agg({"profit_type": "first", "profit": "sum"})
    .rename(columns={"profit": "grand_profit"})
)
display(out)

,profit_type,grand_profit
0,loss,-790.6
1,win,4028.2
2,loss,-575.4


In [434]:
# Create Summary DataFrame
summary = pd.DataFrame(columns=['fast_sma',
                                'slow_sma',
                                
                                'deposit',  # deposit
                                'balance',  # balance
                                
                                'net_profit', # total net profit
                                
                                'gross_profit', # sum of win trades
                                'gross_loss', # sum of fail trades
                                'profit_factor', # ratio (%) => (gross profit / gross loss) => one means (profit = loss)
                                
                                'abs_drawdown', # the largest loss is lower than the initial deposit value
                                'max_drawdown', # maximal loss of the local maximum in the deposit currency and in percents of the deposit
                                'rel_drawdown', #  the maximal loss in percents of the maximum equity value and its corresponding money value
                                
                                'total_trades', # count total trades
                                'total_buy', # count total BUY trades
                                'total_sell', # count total SELL trades
                                
                                'max_win', # Largest profit trade
                                'max_fail', # Largest loss trade
                                
                                'min_lot', # initial lot
                                'lot_multiplier',
                                'max_lot', # highest lot
                                
                                'F',
                                'G'])

# balance =  9971.76
# total net profit =  -28.24
# gross profit =  5.43
# gross loss =  -33.67
# profit factor =  0.16
# largest profit trade =  5.2
# largest loss trade =  -2.2
# total trades =  65
# count SELL trades =  24
# count BUY trades =  41
# count WIN trades =  5
# count LOSS trades =  60
# ave_win_value =  1.09
# ave_loss_value =  -0.56

summary

,fast_sma,slow_sma,deposit,balance,net_profit,gross_profit,gross_loss,profit_factor,abs_drawdown,max_drawdown,...,total_trades,total_buy,total_sell,max_win,max_fail,min_lot,lot_multiplier,max_lot,F,G
